In [1]:
from typing import List, Tuple
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#Parámetros de la simulación del PDF
C_m: float = 1.0
g_Na: float = 120.0  ;g_K: float = 36.0  ;g_L: float = 0.3
E_Na: float = 50.0  ;E_K: float = -77.0  ;E_L: float = -54.387
#Parámetros para establecer el tiempo de la simulación.
dt: float = 0.01  #Intervalos de tiempo para la solución numérica.
tiempo_maximo: float = 70.0  #Tiempo total de la simulación.
#Condiciones de inicio de la simulación
V: float = -65.0  #Voltaje de la membrana (mV)
m: float = 0.05  ;h: float = 0.6 ;n: float = 0.3
I_inyectada = np.zeros(int(tiempo_maximo / dt) + 1)
I_inyectada[500:4000] = 20.0
# Function to simulate the Hodgkin-Huxley model
def Hodgkin_Huxley_simulado(V_0: float, m_0: float, h_0: float, n_0: float, dt: float, tiempo_total: float, I_inyectada: float) -> Tuple[List[float], List[float], List[float], List[float], List[float]]:
    Intervalos_tiempo: List[float] = [0]
    #Guardamos en listas los historiales de las variables para graficar la simulación
    V_historial: List[float] = [V_0];m_historial: List[float] = [m_0];h_historial: List[float] = [h_0];n_historial: List[float] = [n_0]
    #Inicializamos las variables a utilizar por el modelo
    V: float = V_0;m: float = m_0;h: float = h_0;n: float = n_0
    i=0
    while Intervalos_tiempo[-1] < tiempo_total:
        #Calculamos los valores de alfa y beta para encontrar mas adelante los nuevos valores de m, h y n, son las fórmulas del PDF
        a_n: float = (0.01 * (V + 55.0)) / (1.0 - np.exp(-0.1 * (V + 55.0)))
        b_n: float = 0.125 * np.exp(-(V + 65.0) / 80.0)
        a_m: float = (0.1 * (V + 40.0)) / (1.0 - np.exp(-(V + 40.0)/10))
        b_m: float = 4.0 * np.exp(-(V + 65.0) / 18.0)
        a_h: float = 0.07 * np.exp(-(V + 65.0) / 20.0)
        b_h: float = 1.0 / (1.0 + np.exp(-(V + 35.0)/10))
        #Actualizamos el valor de m, h y n
        m += dt * (a_m * (1 - m) - b_m * m)
        h += dt * (a_h * (1 - h) - b_h * h)
        n += dt * (a_n * (1 - n) - b_n * n)
        # Corrientes Iónicas
        I_Na: float = g_Na * m**3 * h * (V - E_Na);I_K: float = g_K * n**4 * (V - E_K);I_L: float = g_L * (V - E_L)
        #Calculamos el voltaje en este instante de tiempo con los datos nuevos
        dV: float = (I_inyectada[i] - I_Na - I_K - I_L) / C_m
        V += dt * dV
        #Actualizamos las listas de historial
        Intervalos_tiempo.append(Intervalos_tiempo[-1] + dt)
        V_historial.append(V);m_historial.append(m);h_historial.append(h);n_historial.append(n)
        i+=1
    return Intervalos_tiempo, V_historial, m_historial, h_historial, n_historial

#Función para graficar la simulación.
def graficar(time_points: List[float], voltage_trace: List[float], m_trace: List[float], h_trace: List[float], n_trace: List[float], I_inyectada_trace: List[float]) -> None:
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)
    fig.add_trace(go.Scatter(x=time_points, y=voltage_trace, mode='lines', name='Voltaje (mV)'), row=1, col=1)
    fig.add_trace(go.Scatter(x=time_points, y=m_trace, mode='lines', name='m', line=dict(dash='dot', color='green')), row=2, col=1)
    fig.add_trace(go.Scatter(x=time_points, y=h_trace, mode='lines', name='h', line=dict(dash='dot', color='red')), row=2, col=1)
    fig.add_trace(go.Scatter(x=time_points, y=n_trace, mode='lines', name='n', line=dict(dash='dot', color='blue')), row=2, col=1)
    fig.add_trace(go.Scatter(x=time_points, y=I_inyectada_trace, mode='lines', name='Corriente Inyectada (mA)', line=dict(dash='dot', color='orange')), row=3, col=1)
    fig.update_xaxes(title_text='Tiempo (ms)', row=2, col=1)
    fig.update_yaxes(title_text='Voltaje (mV)', row=1, col=1)
    fig.update_yaxes(title_text='Variables m, h y n', row=2, col=1)
    fig.update_yaxes(title_text='Corriente Inyectada', row=3, col=1)
    fig.update_layout(title='Modelo Hodgkin-Huxley', showlegend=True)
    fig.show()
#Función main para inicializar la simulación, en este caso decidimos adjuntar un ejemplo.
#Se debe ejecutar la función main para mostrar la gráfica.
def main() -> None:
    Intervalos_tiempo, V_historial, m_historial, h_historial, n_historial = Hodgkin_Huxley_simulado(V, m, h, n, dt, tiempo_maximo, I_inyectada)
    graficar(Intervalos_tiempo, V_historial, m_historial, h_historial, n_historial,I_inyectada)

if __name__ == "__main__":
    main()